In [1]:
from ase.io import iread 

samples=iread('/home/emi/ML/simpleGNN/simplegnn/schnet_test/pressure_test/dataset_with_stress_energy_forces.extxyz', format='extxyz')
sample=next(samples)

sample

Atoms(symbols='O140Si70', pbc=True, cell=[12.5, 12.5, 12.5], calculator=SinglePointCalculator(...))

In [2]:
from simplegnn.painn_calculator import PainnCalculator

In [3]:
from simplegnn.painn import Painn
import torch
cutoff=5.0
model=Painn(natom_basis=60,n_radial=20, cutoff=cutoff,epsilon=1e-5, num_interactions=2)


model_path='/home/emi/ML/simpleGNN/simplegnn/painn_test/painn_model.pth'

model.load_state_dict(torch.load(model_path,weights_only=True))

<All keys matched successfully>

In [4]:
sample.calc=PainnCalculator(model=model, device='cuda', cutoff=cutoff)

In [5]:
sample.get_forces()

array([[-1.9099149e+00,  9.1712415e-01,  3.8962450e+00],
       [-6.8352499e+00,  3.9018422e-01,  9.4471765e-01],
       [-1.2246370e-02,  2.2024596e-01, -3.3597531e+00],
       [-8.5423641e+00,  4.7593814e-01,  1.1568694e+01],
       [ 7.3169351e-01,  1.9645190e+00, -5.6365886e+00],
       [ 2.9832547e+00, -1.7652870e+00,  1.6226465e-01],
       [ 3.6764030e+00,  1.3004155e+00,  2.5058966e+00],
       [-1.9109523e+00,  2.4383519e+00, -2.0771058e+00],
       [-2.5926392e+00,  4.6488967e+00,  9.9950719e-01],
       [ 1.5587907e+00, -1.3740408e+00,  1.0035714e+00],
       [-4.8828506e+00,  7.7952230e-01,  2.9304166e+00],
       [-1.3461962e+00,  2.3254411e+00,  8.8383651e-01],
       [-4.1491213e+00, -1.5661191e+00, -1.3453929e+00],
       [ 1.2979906e+00, -1.4906988e-01,  3.4412358e+00],
       [ 1.9937534e+00,  2.1183982e+00, -4.7579908e+00],
       [-3.4233787e+00, -7.6495540e-01,  2.8138156e+00],
       [-1.3810543e+00,  8.5288572e-01, -3.6268456e+00],
       [-2.1406293e-01, -1.0116

In [6]:
sample.calc.results

{'energy': -796.220703125,
 'forces': array([[-1.9099149e+00,  9.1712415e-01,  3.8962450e+00],
        [-6.8352499e+00,  3.9018422e-01,  9.4471765e-01],
        [-1.2246370e-02,  2.2024596e-01, -3.3597531e+00],
        [-8.5423641e+00,  4.7593814e-01,  1.1568694e+01],
        [ 7.3169351e-01,  1.9645190e+00, -5.6365886e+00],
        [ 2.9832547e+00, -1.7652870e+00,  1.6226465e-01],
        [ 3.6764030e+00,  1.3004155e+00,  2.5058966e+00],
        [-1.9109523e+00,  2.4383519e+00, -2.0771058e+00],
        [-2.5926392e+00,  4.6488967e+00,  9.9950719e-01],
        [ 1.5587907e+00, -1.3740408e+00,  1.0035714e+00],
        [-4.8828506e+00,  7.7952230e-01,  2.9304166e+00],
        [-1.3461962e+00,  2.3254411e+00,  8.8383651e-01],
        [-4.1491213e+00, -1.5661191e+00, -1.3453929e+00],
        [ 1.2979906e+00, -1.4906988e-01,  3.4412358e+00],
        [ 1.9937534e+00,  2.1183982e+00, -4.7579908e+00],
        [-3.4233787e+00, -7.6495540e-01,  2.8138156e+00],
        [-1.3810543e+00,  8.5288572

In [7]:
sample.info

{'source_dir': 'run_0',
 'virial_eV_s6': array([842.4187 , 783.21333, 661.19142, 109.84022, -80.41969,  10.41788]),
 'virial_eV_tensor': array([[842.4187 , 109.84022,  10.41788],
        [109.84022, 783.21333, -80.41969],
        [ 10.41788, -80.41969, 661.19142]]),
 'pressure_kB_s6': array([691.04852, 642.48148, 542.3851 ,  90.10356, -65.96947,   8.54594])}

In [8]:
sample.get_stress()

array([ 0.43302527,  0.40143588,  0.3391175 , -0.04107472,  0.00605576,
        0.05626218])

In [9]:
sample.get_stress()*1602.176634

array([693.78297065, 643.17118995, 543.3261302 , -65.80895548,
         9.7024042 ,  90.14192663])

In [10]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution,Stationary
from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger
from ase import units
from time import perf_counter

In [11]:
# input parameters
time_step    = 1.0    # fsec
num_md_steps = 5000
num_interval = 10
taut         = 1.0    # fs
temperature = 2000    # Kelvin

In [12]:
units.GPa

0.006241509125883258

In [13]:
1/units.GPa

160.21766208

In [15]:
print("temperature = ",temperature)

temperature_str = str(int(temperature)).zfill(4)
output_filename = f"output"
log_filename = output_filename + ".log"
traj_filename = output_filename + ".traj"
print("log_filename = ",log_filename)
print("traj_filename = ",traj_filename)

atoms = sample.copy()
atoms.calc = PainnCalculator(model=model, device='cuda', cutoff=cutoff)

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(atoms, temperature_K=temperature,force_temp=True)
Stationary(atoms)

dyn = NVTBerendsen(atoms, time_step*units.fs, 
                   temperature_K = temperature, taut=taut*units.fs, 
                   loginterval=num_interval, trajectory=traj_filename)


# Print statements
def print_dyn():
    imd = dyn.get_number_of_steps()
    etot  = atoms.get_total_energy()
    temp_K = atoms.get_temperature()
    stress = atoms.get_stress(include_ideal_gas=True)/units.GPa
    stress_ave = (stress[0]+stress[1]+stress[2])/3.0
    elapsed_time = perf_counter() - start_time
    print(f"  {imd: >3}   {etot:.3f}    {temp_K:.2f}    {stress_ave:.2f}  {stress[0]:.2f}  {stress[1]:.2f}  {stress[2]:.2f}  {stress[3]:.2f}  {stress[4]:.2f}  {stress[5]:.2f}    {elapsed_time:.3f}")


dyn.attach(print_dyn, interval=num_interval)
dyn.attach(MDLogger(dyn, atoms, output_filename+".log", header=True, stress=True, peratom=True, mode="a"), interval=num_interval)

# Now run the dynamics
start_time = perf_counter()
print(f"    imd     Etot(eV)    T(K)    stress(mean,xx,yy,zz,yz,xz,xy)(GPa)  elapsed_time(sec)")
dyn.run(num_md_steps)

temperature =  2000
log_filename =  output.log
traj_filename =  output.traj
    imd     Etot(eV)    T(K)    stress(mean,xx,yy,zz,yz,xz,xy)(GPa)  elapsed_time(sec)
    0   -741.932    2000.00    59.71  66.49  61.17  51.46  -6.57  1.00  9.19    0.039
   10   -893.505    3498.18    31.87  33.54  34.57  27.49  0.51  3.27  6.58    0.279
   20   -1040.347    2340.53    22.58  22.68  24.31  20.74  -1.76  1.24  2.61    0.515
   30   -1123.173    2242.93    20.65  19.15  21.79  21.01  -1.02  3.69  -1.08    0.752
   40   -1181.152    2206.20    21.93  22.02  24.49  19.27  2.17  4.52  -0.87    0.988
   50   -1226.234    2107.44    22.30  23.97  23.40  19.53  0.71  1.18  -3.06    1.225
   60   -1252.241    2091.69    21.43  24.53  19.21  20.54  -1.19  2.69  -2.25    1.463
   70   -1279.757    2069.76    19.18  18.36  18.90  20.28  1.18  4.37  -1.36    1.698
   80   -1292.414    2024.37    18.53  17.86  19.71  18.01  1.29  2.59  -0.28    1.933
   90   -1305.273    2057.02    19.25  20.05  19.83  17

True

In [16]:
import nglview as nv
from ase.io import read

frames = read('output.traj', ':')  # 全フレームを読み込み（大きい場合は ':' を '::10' などに）
view = nv.show_asetraj(frames)
view.add_unitcell()            # セルを表示
view.add_spacefill()           # 表示スタイルは好みで（ball+stick など）
view

NGLWidget(max_frame=500)